# Sorting Algorithm Efficiency Prediction

This notebook explores the efficiency of different sorting algorithms on various types of arrays. We aim to train a machine learning model to predict which sorting algorithm performs best based on array characteristics.

## Algorithms Used:
- Bubble Sort
- QuickSort
- Counting Sort
- Radix Sort

We will generate arrays with specific characteristics that favor or disadvantage these algorithms. The model will then learn from the performance data of these algorithms.


In [1]:
def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)
def counting_sort(arr, max_value):
    m = max_value + 1
    count = [0] * m                
    
    for a in arr:
        count[a] += 1             
    i = 0
    for a in range(m):            
        for c in range(count[a]):  
            arr[i] = a
            i += 1
    return arr
def counting_sort_for_radix(input_array, place):
    n = len(input_array)
    output_array = [0] * n
    count_array = [0] * 10

    for i in range(n):
        index = input_array[i] // place
        count_array[index % 10] += 1

    for i in range(1, 10):
        count_array[i] += count_array[i - 1]

    i = n - 1
    while i >= 0:
        index = input_array[i] // place
        output_array[count_array[index % 10] - 1] = input_array[i]
        count_array[index % 10] -= 1
        i -= 1

    for i in range(n):
        input_array[i] = output_array[i]

def radix_sort(arr):
    max_element = max(arr)
    place = 1
    while max_element // place > 0:
        counting_sort_for_radix(arr, place)
        place *= 10


## Data Generation and Performance Measurement

Now, we will generate a diverse set of arrays and measure the performance of each sorting algorithm on these arrays. This data will form the basis of our machine learning model.


In [2]:
import numpy as np

def generate_arrays():
    arrays = []
    for i in range(100): 
        # Arrays for Counting Sort: Small range
        arrays.append(np.random.randint(0, 100, size=1000))

        # Arrays for QuickSort: Poorly-pivoted scenarios
        arrays.append(sorted(np.random.randint(0, 1000, size=1000)))
        
        # Arrays for Radix Sort: Multi-digit numbers
        arrays.append(np.random.randint(0, 100000, size=1000))
        
        # Arrays for Bubble Sort: Large unsorted arrays
        arrays.append(np.random.randint(0, 1000, size=100))

    return arrays


In [3]:
import time

def measure_performance(arrays):
    performance_data = []
    for arr in arrays:
        for sort_function in [quicksort, radix_sort, bubble_sort]:
            start_time = time.time()
            sort_function(arr.copy())  # Use a copy to avoid in-place sorting
            time_taken = time.time() - start_time
            performance_data.append({
                'sort_function': sort_function.__name__,
                'array_length': len(arr),
                'time_taken': time_taken
            })

        # Special case for counting_sort, as it requires the max_value
        max_value = max(arr)
        start_time = time.time()
        counting_sort(arr.copy(), max_value)
        time_taken = time.time() - start_time
        performance_data.append({
            'sort_function': counting_sort.__name__,
            'array_length': len(arr),
            'time_taken': time_taken
        })

    return performance_data


arrays = generate_arrays()
performance_data = measure_performance(arrays)


In [4]:
import pandas as pd

df = pd.DataFrame(performance_data)

# Feature engineering: Add more features based on your array characteristics
# For example, range of values, number of unique values, etc.


## Machine Learning Model Training

We will now train a machine learning model to predict the best sorting algorithm based on the features of the array.


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Prepare the data
X = df.drop(['sort_function', 'time_taken'], axis=1)
y = df['sort_function']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict and evaluate
predictions = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))


Accuracy: 0.203125


## Model Testing and Validation

To test the effectiveness of our model, we will generate new test arrays, predict the best sorting algorithm using our model, and then verify these predictions by actually measuring the performance.


In [6]:
# Generate new test arrays and extract their features
test_arrays = generate_arrays()  # New arrays
test_features = pd.DataFrame([{
    'array_length': len(arr),
    # Add other features you used in the training dataset
    # For example, 'max_value': max(arr) if you used it
    # Make sure to align these features with your training data
} for arr in test_arrays])


predicted_algorithms = model.predict(test_features)

# Compare the predictions with actual performance
# Similar to how you recorded performance data earlier


In [7]:
def compare_performance(test_arrays, predicted_algorithms):
    results = []
    for arr, predicted_algo in zip(test_arrays, predicted_algorithms):
        actual_times = {}
        
        # Measure performance for each algorithm
        for sort_function in [quicksort, counting_sort, radix_sort, bubble_sort]:
            arr_copy = arr.copy()  # Copy to avoid in-place sorting
            start_time = time.time()
            if sort_function == counting_sort:
                sort_function(arr_copy, max(arr_copy))  # Counting sort requires max_value
            else:
                sort_function(arr_copy)
            time_taken = time.time() - start_time
            actual_times[sort_function.__name__] = time_taken

        # Find the algorithm with the actual best performance
        actual_best_algo = min(actual_times, key=actual_times.get)

        # Compare with prediction
        is_correct = predicted_algo == actual_best_algo
        results.append({
            'predicted': predicted_algo,
            'actual': actual_best_algo,
            'is_correct': is_correct
        })

    return results

comparison_results = compare_performance(test_arrays, predicted_algorithms)

# Optionally, print or analyze the comparison results
for result in comparison_results:
    print(result)


{'predicted': 'quicksort', 'actual': 'quicksort', 'is_correct': True}
{'predicted': 'quicksort', 'actual': 'quicksort', 'is_correct': True}
{'predicted': 'quicksort', 'actual': 'quicksort', 'is_correct': True}
{'predicted': 'quicksort', 'actual': 'quicksort', 'is_correct': True}
{'predicted': 'quicksort', 'actual': 'counting_sort', 'is_correct': False}
{'predicted': 'quicksort', 'actual': 'counting_sort', 'is_correct': False}
{'predicted': 'quicksort', 'actual': 'quicksort', 'is_correct': True}
{'predicted': 'quicksort', 'actual': 'quicksort', 'is_correct': True}
{'predicted': 'quicksort', 'actual': 'quicksort', 'is_correct': True}
{'predicted': 'quicksort', 'actual': 'counting_sort', 'is_correct': False}
{'predicted': 'quicksort', 'actual': 'quicksort', 'is_correct': True}
{'predicted': 'quicksort', 'actual': 'quicksort', 'is_correct': True}
{'predicted': 'quicksort', 'actual': 'quicksort', 'is_correct': True}
{'predicted': 'quicksort', 'actual': 'quicksort', 'is_correct': True}
{'pre

In [8]:
def compare_performance(test_arrays, predicted_algorithms):
    correct_predictions = 0
    total_predictions = len(test_arrays)

    for arr, predicted_algo in zip(test_arrays, predicted_algorithms):
        actual_times = {}
        
        for sort_function in [quicksort, counting_sort, radix_sort, bubble_sort]:
            arr_copy = arr.copy()
            start_time = time.time()
            if sort_function == counting_sort:
                sort_function(arr_copy, max(arr_copy))
            else:
                sort_function(arr_copy)
            time_taken = time.time() - start_time
            actual_times[sort_function.__name__] = time_taken

        actual_best_algo = min(actual_times, key=actual_times.get)

        if predicted_algo == actual_best_algo:
            correct_predictions += 1

    return correct_predictions / total_predictions * 100  # Return percentage

percent_correct_predictions = compare_performance(test_arrays, predicted_algorithms)
print(f"Percent of Correct Predictions: {percent_correct_predictions}%")


Percent of Correct Predictions: 88.25%


## Conclusions and Future Improvements

The model's ability to predict the best sorting algorithm showcases the potential of machine learning in algorithmic efficiency analysis. However, there are several areas for future improvement:

- **Feature Expansion**: Including more features like the degree of sorting and the number of unique elements in the array could improve the model's accuracy.
- **Algorithm Optimization**: The sorting algorithms can be further optimized for better performance, especially for larger datasets.
- **Model Complexity**: Exploring more complex models or fine-tuning the current model could yield better predictive accuracy.

### Potential Use Cases

This model can be useful in scenarios where sorting is a frequent operation, and the nature of the data varies significantly. It can be integrated into software systems to dynamically choose the most efficient sorting algorithm based on the data it encounters.
